In [2]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install peft
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset = load_dataset('csv', data_files='drive/My Drive/ECE1786_data/clean_data_gpt2.csv')


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'essay', 'label'],
        num_rows: 9766
    })
})


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    combined_text = examples["prompt"] + '\n' + examples["essay"]
    return tokenizer(combined_text, padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/9766 [00:00<?, ? examples/s]

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000, 7000))
small_test_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(7000, 9766))


In [9]:
print(small_train_dataset)

Dataset({
    features: ['prompt', 'essay', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=12)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,2.302000,2.330951,0.159000
2,2.318900,2.203277,0.183000
3,2.276300,2.246374,0.192500
4,2.089600,2.362928,0.187500
5,1.987500,2.564688,0.211500
6,1.787400,2.931081,0.220500
7,1.824300,3.183874,0.218500
8,1.656800,3.744631,0.218000
9,1.629500,4.571116,0.213000
10,1.758000,4.956617,0.213000


TrainOutput(global_step=50000, training_loss=1.9431987927246093, metrics={'train_runtime': 3795.4182, 'train_samples_per_second': 13.174, 'train_steps_per_second': 13.174, 'total_flos': 2.61320343552e+16, 'train_loss': 1.9431987927246093, 'epoch': 10.0})

In [17]:
model.save_pretrained("drive/My Drive/ECE1786_data/trained_model")
tokenizer.save_pretrained("drive/My Drive/ECE1786_data/trained_model")

('drive/My Drive/ECE1786_data/tokenizer_config.json',
 'drive/My Drive/ECE1786_data/special_tokens_map.json',
 'drive/My Drive/ECE1786_data/vocab.json',
 'drive/My Drive/ECE1786_data/merges.txt',
 'drive/My Drive/ECE1786_data/added_tokens.json',
 'drive/My Drive/ECE1786_data/tokenizer.json')

In [19]:
# Set up evaluation arguments
eval_args = TrainingArguments(
    per_device_eval_batch_size=1,
    output_dir="./results",
    logging_dir="./logs",
)

# Initialize Trainer with test dataset and custom compute metrics
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics
)

# Run evaluation
trainer.evaluate()


{'eval_loss': 5.086670875549316,
 'eval_model_preparation_time': 0.0024,
 'eval_accuracy': 0.20535068691250905,
 'eval_runtime': 51.8354,
 'eval_samples_per_second': 53.361,
 'eval_steps_per_second': 53.361}

In [20]:
def compute_metrics_tol(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy with a tolerance of ±1 around the actual label
    correct = (predictions == labels) | (predictions == labels + 1) | (predictions == labels - 1)
    accuracy = np.mean(correct)  # Average to get the accuracy

    return {"accuracy": accuracy}

In [22]:
# Set evaluation arguments
eval_args = TrainingArguments(
    per_device_eval_batch_size=1,
    output_dir="./results",
)

# Initialize Trainer with test dataset and compute_metrics function
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics_tol
)

# Run evaluation on the test set
results = trainer.evaluate()

# Output test accuracy
print("Test Set Accuracy:", results["eval_accuracy"])

Test Set Accuracy: 0.47107736804049166


In [31]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the fine-tuned model and tokenizer
save_directory = "drive/My Drive/ECE1786_data/trained_model"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# Define a function to classify a single example
def classify_single_input(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Pass the tokenized input through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class (index of the highest logit)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    return predicted_class

# Example input for classification
input_text = "While many people go to university for academic study, more people should be encouraged to do vocational training because there is a lack of qualified workers such as electricians and plumbers. Do you agree or disagree?" + "\n" + "Nowadays, with the development of education, more and more students go to colleges for academic learning. However, some people consider that we should provide more vocational courses because a lack of practical workers such as electricians and plumbers. Personally, I think academic study is obviously important bacause it can improve the technology and the misunderstanding of job-not the lack of vocational training- is the main reason for a lack of labored workers.\
It is no doubt that the academic learning in university can induct the practical training and provoke the development of technology in different areas such as agricuture. If we deeply understand the theory of academic knowledge and reasonablely apply them in different fields, we could efficiently improve the technology and help human beings to increase their living standard. For instance, the development of drones turly improve the quantity of crops, which is benifit for famers. However, this technology need a lot of basic theories to invent the drones and teach worker to handle this machine. At this time, the academic study becomes much more important for practical workers.\
Some people claim that it is because a lack of vocational training, thus leading to a lack of qualified worker. However, in my opinion, I think there are mainly two reasons come from the misunderstanding of jod cause this bad situation. For one aspect, some people think that practical workers usually cannot earn much more money compared to empolyees working in office. Thus, they just give up this occupation for their career. For another aspect, some people do not respect those qualified worker, which make them feel their job are secondary. At last, there are no more people would like to choose this type of works.\
In conclusion, I think academic study is very much important for students to improve the technology, and the lack of practical workers is not because of less provided vocational training, but the misunderstanding of job between some people."
# Get the predicted class
predicted_class = classify_single_input(input_text)
print(f"Predicted Class: {predicted_class}")

Predicted Class: 0
